In [12]:
%reload_ext autoreload
%autoreload 2
import os
from pathlib import Path
import numpy as np
import pandas as pd
from aldiscore.prediction import utils
from aldiscore import ROOT, RSTATE
from aldiscore.constants.constants import STAT_SEP

In [13]:
data_dir = Path("/hits/fast/cme/bodynems/data/paper")
feat_df, drop_df, label_df = utils.load_features(
    data_dir,
    exclude_features=["is_dna", "num_seqs", "seq_length"],
)

print(feat_df.shape)
print(drop_df.shape)
print(label_df.shape)

Dropping 0 NaN rows...
(11431, 397)
(11431, 21)
(11431, 1)


In [14]:
feat_df.columns[feat_df.columns.str.contains("ratio")]

Index(['min.psa_score_ratio', 'max.psa_score_ratio', 'mean.psa_score_ratio',
       'std.psa_score_ratio', 'p1.psa_score_ratio', 'p5.psa_score_ratio',
       'p10.psa_score_ratio', 'p20.psa_score_ratio', 'p30.psa_score_ratio',
       'p40.psa_score_ratio', 'p50.psa_score_ratio', 'p60.psa_score_ratio',
       'p70.psa_score_ratio', 'p80.psa_score_ratio', 'p90.psa_score_ratio',
       'p95.psa_score_ratio', 'p99.psa_score_ratio', 'iqr.psa_score_ratio',
       'min.psa_gap_ratio', 'max.psa_gap_ratio', 'mean.psa_gap_ratio',
       'std.psa_gap_ratio', 'p1.psa_gap_ratio', 'p5.psa_gap_ratio',
       'p10.psa_gap_ratio', 'p20.psa_gap_ratio', 'p30.psa_gap_ratio',
       'p40.psa_gap_ratio', 'p50.psa_gap_ratio', 'p60.psa_gap_ratio',
       'p70.psa_gap_ratio', 'p80.psa_gap_ratio', 'p90.psa_gap_ratio',
       'p95.psa_gap_ratio', 'p99.psa_gap_ratio', 'iqr.psa_gap_ratio',
       'min.psa_stretch_ratio', 'max.psa_stretch_ratio',
       'mean.psa_stretch_ratio', 'std.psa_stretch_ratio',
       'p1.

In [15]:
from sklearn.model_selection import train_test_split

train_idxs, test_idxs = train_test_split(
    feat_df.index.to_list(), test_size=0.2, random_state=RSTATE
)
test_idxs, valid_idxs = train_test_split(test_idxs, test_size=0.5, random_state=RSTATE)
print(len(train_idxs), len(test_idxs), len(valid_idxs))
#
X_train = feat_df.loc[train_idxs]
X_test = feat_df.loc[test_idxs]
X_valid = feat_df.loc[valid_idxs]
y_train = label_df.loc[train_idxs].iloc[:, 0]
y_test = label_df.loc[test_idxs].iloc[:, 0]
y_valid = label_df.loc[valid_idxs].iloc[:, 0]

9144 1143 1144


In [ ]:
from aldiscore.prediction.predictor import DifficultyPredictor
import lightgbm as lgb

model = DifficultyPredictor("latest").model

feat_df = feat_df[model.feature_name()]
imps = model.feature_importance("gain")
lgb.plot_importance(model, figsize=(6, 7), importance_type="gain", max_num_features=30)

array([8.71390769e+00, 1.54115174e+00, 8.89568865e+00, 2.51535147e+00,
       9.25852479e-01, 7.87809719e-01, 1.60882444e+00, 2.25045320e+00,
       4.81284503e+00, 1.84644547e+00, 1.83868243e+00, 1.80623852e+00,
       2.40535260e+00, 1.80810842e+00, 3.36660644e+00, 2.41301713e+00,
       5.18274235e+00, 3.18864625e+00, 1.76589650e+00, 9.07197822e-01,
       2.13972561e+00, 1.56438680e+00, 1.90544314e+00, 1.24849591e+00,
       1.70157907e+00, 1.23264516e+00, 6.29945565e+00, 2.93275027e+00,
       4.32810163e+00, 2.27504934e+00, 1.14210718e+00, 3.46769623e+00,
       7.77152218e-01, 3.78000408e+00, 3.22073637e+00, 1.56184134e+00,
       1.93123208e+00, 1.20779513e+00, 1.51582323e+00, 1.05775860e+00,
       1.25162721e+00, 7.67812543e-01, 9.77880969e-01, 1.00195966e+00,
       8.11051651e-01, 1.22910049e+00, 1.76663973e+00, 7.03999168e-01,
       9.48000374e-01, 1.26314893e+00, 1.39039091e+00, 5.00785517e-01,
       1.75778127e+00, 2.63643534e+00, 1.20931861e+00, 1.37904535e+00,
      

In [37]:
imp_df = pd.concat([pd.Series(model.feature_name()), pd.Series(imps)], axis=1)
imp_df.columns = ["name", "gain"]
imp_df
imp_df["group"] = pd.Series(imp_df.name.str.split(".").map(lambda v: v[-1]))
imp_df.sort_values("gain", ascending=False)
print(set(imp_df.group))
for pat in [
    "mean",
    "min",
    "max",
    "mean",
    "iqr",
    "p50",
    "std",
    # "count",
    # "len",
    # "len_logdiff",
]:
    imp_df.group = imp_df.group.str.rsplit("_" + pat).map(lambda v: v[0])
groups = set(imp_df.group)
groups

{'psa_tc_mean', '10-mer_js', 'psa_gap_len_std', '4-mer_ent', 'psa_tc_min', 'hpoly_js_count', 'hpoly_js_len', '10-mer_ent', 'psa_gap_len_mean', 'psa_tc_max', 'psa_tc_p50', 'psa_gap_ratio', 'char_js', 'psa_tc_std', '4-mer_js', '7-mer_ent', 'psa_stretch_ratio', 'lbgp', '13-mer_js', '13-mer_ent', 'char_ent', 'psa_score_ratio', '7-mer_js'}


{'10-mer_ent',
 '10-mer_js',
 '13-mer_ent',
 '13-mer_js',
 '4-mer_ent',
 '4-mer_js',
 '7-mer_ent',
 '7-mer_js',
 'char_ent',
 'char_js',
 'hpoly_js_count',
 'hpoly_js_len',
 'lbgp',
 'psa_gap_len',
 'psa_gap_ratio',
 'psa_score_ratio',
 'psa_stretch_ratio',
 'psa_tc'}

In [38]:
imp_df.loc[imp_df.name.str.contains("gap_len")].sort_values(
    "gain", ascending=False
).iloc[:20]

,name,gain,group
221,p1.psa_gap_len_mean,40.761726,psa_gap_len
226,p40.psa_gap_len_mean,14.583022,psa_gap_len
225,p30.psa_gap_len_mean,10.504610,psa_gap_len
241,p10.psa_gap_len_std,9.665584,psa_gap_len
230,p80.psa_gap_len_mean,9.420269,psa_gap_len
240,p5.psa_gap_len_std,9.370055,psa_gap_len
249,p90.psa_gap_len_std,8.337445,psa_gap_len
223,p10.psa_gap_len_mean,6.941854,psa_gap_len
242,p20.psa_gap_len_std,6.613076,psa_gap_len
228,p60.psa_gap_len_mean,6.107592,psa_gap_len


In [39]:
stat_df = imp_df.drop("name", axis=1).groupby("group").aggregate(["sum", "count"])
T_GAIN = "Gain (%)"
# M_GAIN = "mean_gain (%)"
stat_df.columns = [T_GAIN, "Count"]
stat_df = stat_df.sort_values(T_GAIN, ascending=False)
gain_cols = [T_GAIN]
stat_df[gain_cols] = (stat_df[gain_cols] / stat_df[gain_cols].sum(axis=0)).round(
    4
) * 100
stat_df

,Gain (%),Count
group,,
psa_tc,76.30,90
psa_score_ratio,5.55,18
psa_gap_ratio,4.47,18
psa_gap_len,3.46,36
13-mer_ent,1.05,18
10-mer_js,1.03,18
13-mer_js,1.00,18
char_ent,0.90,18
7-mer_js,0.88,18


In [34]:
# name_map = dict(
#     tc_base="psa_tc",
# )
# for i in [5, 7, 9, 11]:
#     for sfx in ["ent", "js"]:
#         name_map[f"{i}mer_{sfx}"] = f"{i}-mer_{sfx}"
# name_map["js_char"] = "char_js"
# name_map["ent_char"] = "char_ent"
# name_map["js_hpoly_len"] = "hpoly_js_len"
# name_map["js_hpoly_count"] = "hpoly_js_count"
# stat_df = stat_df.rename(name_map)

In [40]:
stat_df["Description"] = ""

stat_df.at["psa_tc", "Description"] = "Transitive consistency of PSA triplets."
stat_df.at["psa_score_ratio", "Description"] = (
    "Alignment score scaled by the minimum sequence length."
)
stat_df.at["psa_gap_ratio", "Description"] = (
    "Number of gaps divided by total number of characters."
)
stat_df.at["psa_gap_len", "Description"] = (
    "Features based on the lengths of gap regions."
)
stat_df.at["char_ent", "Description"] = "Shannon entropy of character distributions."
stat_df.at["char_js", "Description"] = (
    "Pairwise Jensen-Shannon divergence of character distributions."
)
stat_df.at["psa_stretch_ratio", "Description"] = (
    "Ratio between max. sequence length and alignment length."
)
stat_df.at["hpoly_js_len", "Description"] = (
    "Pairwise Jensen-Shannon divergence of homopolymer length distributions."
)
stat_df.at["hpoly_js_count", "Description"] = (
    "Pairwise Jensen-Shannon divergence of homopolymer count distributions."
)
stat_df.at["lbgp", "Description"] = (
    "Lower bound on the gap percentage (1 - mean_len/max_len)."
)
for i in [4, 7, 10, 13]:
    stat_df.at[f"{i}-mer_js", "Description"] = (
        "Pairwise Jensen-Shannon divergence of k-mer distributions."
    )
    stat_df.at[f"{i}-mer_ent", "Description"] = (
        "Shannon entropy of k-mer distributions."
    )
stat_df
print(stat_df.to_latex(escape=True, float_format="%.2f"))

\begin{tabular}{lrrl}
\toprule
 & Gain (\%) & Count & Description \\
group &  &  &  \\
\midrule
psa\_tc & 76.30 & 90 & Transitive consistency of PSA triplets. \\
psa\_score\_ratio & 5.55 & 18 & Alignment score scaled by the minimum sequence length. \\
psa\_gap\_ratio & 4.47 & 18 & Number of gaps divided by total number of characters. \\
psa\_gap\_len & 3.46 & 36 & Features based on the lengths of gap regions. \\
13-mer\_ent & 1.05 & 18 & Shannon entropy of k-mer distributions. \\
10-mer\_js & 1.03 & 18 & Pairwise Jensen-Shannon divergence of k-mer distributions. \\
13-mer\_js & 1.00 & 18 & Pairwise Jensen-Shannon divergence of k-mer distributions. \\
char\_ent & 0.90 & 18 & Shannon entropy of character distributions. \\
7-mer\_js & 0.88 & 18 & Pairwise Jensen-Shannon divergence of k-mer distributions. \\
7-mer\_ent & 0.83 & 18 & Shannon entropy of k-mer distributions. \\
10-mer\_ent & 0.80 & 18 & Shannon entropy of k-mer distributions. \\
char\_js & 0.78 & 18 & Pairwise Jensen-Shannon 

In [22]:
# Feature classes:
# - tc_base :
# - psa_score_ratio : Alignment score scaled by the minimum sequence length
# - kmer_js : Pairwise Jensen-Shannon divergence of kmer distributions
# - kmer_ent : Entropy of kmer distributions
# - lbgp : Lower bound on the percentage of gaps
# - psa_gap :